In [1]:
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime

In [13]:
# Функция для получения курса валюты из XML-данных ЦБР
def get_currency_rates(currency_id, year):
    url = f"https://www.cbr.ru/scripts/XML_dynamic.asp?date_req1=01/01/{year}&date_req2=31/12/{year}&VAL_NM_RQ={currency_id}"
    r = requests.get(url)
    r.encoding = 'windows-1251'  # Кодировка ЦБР
    df = pd.read_xml(BytesIO(r.content))
    return df


In [14]:
# Валюты и их ID в ЦБР
currency_ids = {
    'USD': 'R01235',
    'EUR': 'R01239',
    'CNY': 'R01375'
}

In [57]:
dfs = []
year = 2023

for curr, curr_id in currency_ids.items():
    df = get_currency_rates(curr_id, year)
    df['Valute'] = curr
    dfs.append(df)

In [58]:
# Объединяем все в один датафрейм
df = pd.concat(dfs, ignore_index=True)

In [59]:
# Переименование колонок, преобразования
df.rename(columns={'Date': 'Дата', 'Value': 'Курс', 'Value': 'Курс', 'Valute': 'Валюта', 'VunitRate': 'Значение курса'}, inplace=True)
df

,Дата,Id,Nominal,Курс,Значение курса,Валюта
0,10.01.2023,R01235,1,"70,3002","70,3002",USD
1,11.01.2023,R01235,1,"69,6094","69,6094",USD
2,12.01.2023,R01235,1,"69,0202","69,0202",USD
3,13.01.2023,R01235,1,"67,7775","67,7775",USD
4,14.01.2023,R01235,1,"67,5744","67,5744",USD
...,...,...,...,...,...,...
736,26.12.2023,R01375,1,"12,8379","12,8379",CNY
737,27.12.2023,R01375,1,"12,7937","12,7937",CNY
738,28.12.2023,R01375,1,"12,8011","12,8011",CNY
739,29.12.2023,R01375,1,"12,6749","12,6749",CNY


In [60]:
# В колонке 'Курс' содержится значение с запятой, заменим запятую на точку и преобразуем в float
df['Курс'] = df['Курс'].str.replace(',', '.').astype(float)
df['Значение курса'] = df['Значение курса'].str.replace(',', '.').astype(float)

In [61]:
# Парсим дату
df['Дата'] = pd.to_datetime(df['Дата'], format='%d.%m.%Y')

In [67]:
# Добавляем нужные колонки
df['Дата начала месяца'] = df['Дата'].values.astype('datetime64[M]')
df['Год'] = df['Дата'].dt.year
df['Квартал'] = df['Дата'].dt.quarter
df['Месяц'] = df['Дата'].dt.month
df['Неделя года'] = df['Дата'].dt.isocalendar().week
df['Значение курса'] = df['Курс']

In [69]:
# Оставляем только нужные колонки
df = df[['Дата', 'Дата начала месяца', 'Год', 'Квартал', 'Месяц', 'Неделя года', 'Курс', 'Значение курса', 'Валюта']]
df

,Дата,Дата начала месяца,Год,Квартал,Месяц,Неделя года,Курс,Значение курса,Валюта
0,2023-01-10,2023-01-01,2023,1,1,2,70.3002,70.3002,USD
1,2023-01-11,2023-01-01,2023,1,1,2,69.6094,69.6094,USD
2,2023-01-12,2023-01-01,2023,1,1,2,69.0202,69.0202,USD
3,2023-01-13,2023-01-01,2023,1,1,2,67.7775,67.7775,USD
4,2023-01-14,2023-01-01,2023,1,1,2,67.5744,67.5744,USD
...,...,...,...,...,...,...,...,...,...
736,2023-12-26,2023-12-01,2023,4,12,52,12.8379,12.8379,CNY
737,2023-12-27,2023-12-01,2023,4,12,52,12.7937,12.7937,CNY
738,2023-12-28,2023-12-01,2023,4,12,52,12.8011,12.8011,CNY
739,2023-12-29,2023-12-01,2023,4,12,52,12.6749,12.6749,CNY


In [71]:
df

,Дата,Дата начала месяца,Год,Квартал,Месяц,Неделя года,Курс,Значение курса,Валюта
0,2023-01-10,2023-01-01,2023,1,1,2,70.3002,70.3002,USD
1,2023-01-11,2023-01-01,2023,1,1,2,69.6094,69.6094,USD
2,2023-01-12,2023-01-01,2023,1,1,2,69.0202,69.0202,USD
3,2023-01-13,2023-01-01,2023,1,1,2,67.7775,67.7775,USD
4,2023-01-14,2023-01-01,2023,1,1,2,67.5744,67.5744,USD
...,...,...,...,...,...,...,...,...,...
736,2023-12-26,2023-12-01,2023,4,12,52,12.8379,12.8379,CNY
737,2023-12-27,2023-12-01,2023,4,12,52,12.7937,12.7937,CNY
738,2023-12-28,2023-12-01,2023,4,12,52,12.8011,12.8011,CNY
739,2023-12-29,2023-12-01,2023,4,12,52,12.6749,12.6749,CNY
